# Trading Manager for Crypto

This notebook allow to comunicate with the services running in real time. All the comunications it is with MQ messages.

In [1]:
from src.infrastructure.database_handler import Universe
from src.application import conf
from src.domain.models.trading.petition import Petition
from src.domain.models.trading.order import Order
import datetime as dt
from src.infrastructure.brokerMQ import Emit_Events
import time
import math

def round_decimals_down(number:float, decimals:int=2):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor

def round_decimals_up(number:float, decimals:int=2):
    """
    Returns a value rounded up to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.ceil(number)

    factor = 10 ** decimals
    return math.ceil(number * factor) / factor

# Conection to the database
store = Universe(host=conf.MONGO_HOST, port=conf.MONGO_PORT)
lib_petitions = store.get_library('petitions')
# Conection to broker mq
config_brokermq = {'host': conf.RABBITMQ_HOST, 'port': conf.RABBITMQ_PORT, 'user': conf.RABBITMQ_USER,
                   'password': conf.RABBITMQ_PASSWORD}
emiter = Emit_Events(config=self.config_brokermq)

def get_data_petition(name):
    for i in range(10):
        time.sleep(1)
        # check if name in list_symbols
        list_symbols = lib_petitions.list_symbols()
        if name in list_symbols:
            return lib_petitions.read(name).data
        time.sleep(5)



/usr/local/lib/python3.8/site-packages/arctic/store/_pandas_ndarray_store.py:8: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


ValueError: Port must be an integer between 0 and 65535: None

## Analice Realtime positions from the portfolio VS Realtime position in Exchange
It sends a petition msg to the portfolio service to get the realtime positions.

In [12]:
# Create the petition
name_to_saving = f'positions_{dt.datetime.now().strftime("%Y%m%d%H%M%S")}'
function_to_run='get_saved_values_strategies_last' #get_saved_values_strategy  
# name of portfolio which we want to know positions
name_portfolio = 'PortfolioForex1'
petition_pos = Petition(datetime=dt.datetime.now(), function_to_run=function_to_run,
                               name_to_saving=name_to_saving, name_portfolio=name_portfolio)

emiter.publish_event('petition', petition_pos)


In [16]:
# Get data Petition
data_petition = get_data_petition(name_to_saving)
## Agregate data_petition by ticker base
agregate = {}
tickers = []
for _id in data_petition.keys():
    p = data_petition[_id]
    t = data_petition[_id]['ticker']
    if t not in tickers:
       tickers.append(t)
    t1 = t.split('-')[0]
    t2 = t.split('-')[1]
    agregate.setdefault(t1,0)
    agregate.setdefault(t2,0)
    agregate[t1] += p['position']*p['quantity']
    if p['position'] == 0:
        agregate[t2] += abs((p['position'] -1 )*p['quantity']*p['close'])

print('Porfolio Potitions')
list_currency = [c.split('-')[0] for c in agregate.keys() ]
agregate

Porfolio Potitions


{'BTC': 1, 'USDT': 0, 'ETH': 1}

In [ ]:
# Get position in Broker or Exchange
broker = 'kucoin'
if broker in ['kucoin', 'binance']:
    from src.infraestructure.crypto.exchange_handler import Trading
else:
    pass

trading = Trading(exchange_or_broker=broker,
                  config_brokermq=config_brokermq)
_broker_pos =trading.get_accounts()

broker_pos = {c['currency']:float(c['balance']) for c in _broker_pos if c['currency'] in list_currency}
broker_pos

In [71]:
# Compute diference between broker and portfolio
diference = {}
for c in broker_pos.keys():
    diference[c]  = agregate[c] - broker_pos[c]
    if diference[c] > 0:
        diference[c] = round_decimals_down(diference[c],4)
    else:
        diference[c] = round_decimals_up(diference[c],4)
print(diference)

{'USDT': -193.6037, 'ETH': 0.1}


In [72]:
# Create order to close diference
n= 0
for t in tickers:
   d = diference[t.split('-')[0]]
   if abs(d) > 0.01:
       if d > 0:
           action = 'buy'
       else:
           action = 'sell'
       order_id_sender =f' {0}_{n}_{dt.datetime.utcnow().strftime("%Y%m%d%H%M%S")}'
       order = Order(datetime=dt.datetime.utcnow(),
                            ticker=t, action=action,
                            price=0, quantity=abs(d), type='market', sender_id=0,
                            order_id_sender=order_id_sender)
       emiter.publish_event('order', order)
       n += 1

